# PINT Example Session

The PINT homepage is at:  https://github.com/nanograv/PINT.
There, you can find a Wiki with information on installing PINT
PINT can be run via a script, in an interactive session with ipython or jupyter, or using one of the command-line tools provided.

## Times of Arrival (TOAs)

The raw data for PINT are TOAs, which can be read in from files in a variety of formats, or constructed programatically. PINT currently can read TEMPO, Tempo2, and Fermi "FT1" photon files.

Note:  The first time TOAs get read in, lots of processing (can) happen, which can take some time. However, a  "pickle" file is saved, so the next time the same file is loaded (if nothing has changed), the TOAs will be loaded from the pickle file, which is much faster.

In [1]:
from __future__ import print_function, division
import numpy as np
import astropy.units as u

In [2]:
# Here is how to create a single TOA in Python
import pint.toa as toa
a = toa.TOA((54567, 0.876876876876876), 4.5, freq=1400.0, obs="GBT", backend="GUPPI",location=None)
print(a)

54567.876876876876876:  4.500 us error from 'gbt' at 1400.0000 MHz {'location': None, 'backend': 'GUPPI'}


In [3]:
# An example of reading a TOA file
import pint.toa as toa
t = toa.get_TOAs("NGC6440E.tim",usepickle=False)

INFO: Applying clock corrections. [pint.toa]
INFO: Observatory gbt, loading clock file /Users/jingluo/Research_codes/PINT/pint/datafiles/time.dat [pint.observatory.topo_obs]
INFO: Observatory gbt, loading GPS clock file /Users/jingluo/Research_codes/PINT/pint/datafiles/gps2utc.clk [pint.observatory.topo_obs]
INFO: Observatory gbt, loading BIPM clock file /Users/jingluo/Research_codes/PINT/pint/datafiles/tai2tt_bipm2015.clk [pint.observatory.topo_obs]
INFO: Getting IERS params and computing TDBs. [pint.toa]
INFO: Computing TDB columns. [pint.toa]
INFO: Computing observatory positions and velocities. [pint.toa]
INFO: Compute positions and velocities of observatories and Earth (planets = False), using DE421 ephemeris [pint.toa]
INFO: Adding columns ssb_obs_pos ssb_obs_vel obs_sun_pos [pint.toa]


In [4]:
#  Here is a summary.
t.print_summary()

Number of TOAs:  62
Number of commands:  1
Number of observatories:  1 ['gbt']
MJD span:  53478.286 to 54187.587
gbt TOAs (62):
  Min error:     13.2 us
  Max error:     118 us
  Mean error:    26.9 us
  Median error:  22.1 us
  Error stddev:  15.6 us



In [5]:
# Here is the MJD of the first TOA
t.get_mjds()[0]

53478.285871419219

TOAs are stored in a [Astropy Table](http://astropy.readthedocs.org/en/latest/table/)  in an instance of the TOAs class. 

In [6]:
# List the table columns, which include pre-computed TDB times and solar system positions and velocities
t.table.colnames

['index',
 'mjd',
 'mjd_float',
 'error',
 'freq',
 'obs',
 'flags',
 'tdb',
 'tdbld',
 'ssb_obs_pos',
 'ssb_obs_vel',
 'obs_sun_pos']

Lots of cool things that tables can do...

In [7]:
# This pops open a browser window showing the contents of the table 
tt = t.table
tt.show_in_browser()

Can do fancy sorting, selecting, re-arranging very easily.

In [8]:
select = tt['error'] < 20*u.us
print(select)

[False False False False False False False  True False False False False
 False False  True False  True False False False  True False  True False
  True  True  True  True False  True False  True  True  True False False
 False False False False False False  True  True False  True  True False
 False False  True False False False False False False False False False
 False False]


In [9]:
tt['tdb'][select]

53679.876388
53690.851265
53695.8596507
53709.8103552
53740.7753531
53801.5921875
53833.2985648
53833.5032122
53843.3328338
53865.3767058
53895.3242239


Many PINT routines / classes / functions use [Astropy Units](http://astropy.readthedocs.org/en/latest/units/) internally or externally:

In [10]:
t.get_errors() 

21.71
21.95
29.95
25.46
23.43
31.67
30.26
13.52
21.64
27.41
24.58


The times in each row contain (or are derived from) [Astropy Time](http://astropy.readthedocs.org/en/latest/time/) objects:

In [11]:
t0 = tt['mjd'][0]

In [12]:
t0.tai

<Time object: scale='tai' format='pulsar_mjd' value=53478.2862418>

But the most useful timescale, TDB is also stored as long double numpy arrays, to maintain precision:

In [13]:
tt['tdbld'][:3]

53478.2866143
53483.2774481
53489.4691327


## Timing (or other) Models

Now let's define and load a timing model

In [14]:
import pint.models as models
m = models.StandardTimingModel()
m.read_parfile("NGC6440E.par")

TypeError: 'TimingModel' object is not callable

In [ ]:
print(m.as_parfile())

Timing models are basically composed of "delay" terms and "phase" terms. Currently the delay terms are organized into two 'levels'. L1 are delay terms local to the Solar System, which are needed for computing 'barycenter-corrected' TOAs. L2 are delay terms for the binary system.  (This system may change in the future to accommodate more complicated scenarios)

In [ ]:
m.delay_funcs

In [ ]:
m.phase_funcs

Can easily show/compute individual terms...

In [ ]:
ds = m.solar_system_shapiro_delay(tt)
print(ds)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(t.get_mjds(high_precision=False), ds*1e6, 'x')
plt.xlabel("MJD") ; plt.ylabel("Delay ($\mu$s)")

or all of the terms added together:

In [ ]:
m.delay(tt)

In [ ]:
m.phase(tt)

## Residuals

In [ ]:
import pint.residuals as r

In [ ]:
rs = r.resids(t, m).phase_resids

In [ ]:
# The get_mjds() function returns float MJDs for easy plotting, rather than astropy Time objects
plt.plot(t.get_mjds(), rs, 'x')
plt.title("%s Pre-Fit Timing Residuals" % m.PSR.value)
plt.xlabel('MJD'); plt.ylabel('Residual (phase)')
plt.grid()


## Fitting and Post-Fit residuals

The fitter is *completely* separate from the model and the TOA code.  So you can use any type of fitter with some easy coding.  This example uses a very simple Powell minimizer from the SciPy optimize module. 

In [ ]:
import pint.fitter as fit
f = fit.WlsFitter(t, m)
f.fit_toas()

In [ ]:
print("Best fit has reduced chi^2 of", f.resids.chi2_reduced)
print("RMS in phase is", f.resids.phase_resids.std())
print("RMS in time is", f.resids.time_resids.std().to(u.us))
print("\n Best model is:")
print(f.model.as_parfile())


In [ ]:
plt.errorbar(t.get_mjds(),
             f.resids.time_resids.to(u.us).value,
             t.get_errors().to(u.us).value, fmt='x')
plt.title("%s Post-Fit Timing Residuals" % m.PSR.value)
plt.xlabel('MJD'); plt.ylabel('Residual (us)')
plt.grid()

## Other interesting things

We can make Barycentered TOAs in a single line!

In [ ]:
m.get_barycentric_toas(tt)